In [2]:
import os
import glob
import shutil
import csv
import yaml
import re
from finetune_YOLO import YOLOFinetuner
from evaluate_YOLO import YOLOEvaluator
root_dir = '/home/wandb-runs/pace-v2/'

In [ ]:
sweep_dir = '/home/wandb-runs/pace-v2/2D_CNP-only-50/yejjmzte'
run = 'confused-sweep-14'

args = yaml.safe_load(open(os.path.join(sweep_dir, run, 'args.yaml')))
args['epochs'] = 50
args['patience'] = 20
args['name'] = args['name'] + '_extended'
args['save_dir'] = os.path.join(sweep_dir, args['name'])
args['data'] = '/home/data/configs/pace_v2.yaml'
args['project'] = sweep_dir.replace('pace-v2/', 'pace-v2-extended/')

# ############################### TEMP #######################################################################
# args['epochs'] = 2
# args['fraction'] = 0.001

yolo_finetuner = YOLOFinetuner(**args)
results_train = yolo_finetuner.train_model()

# Sweep processing

In [2]:
## Check which run within a given sweep set is missing a val dir which contains a yaml referring to it
## to do this, first go through all the val dirs in that sweep set and check their yaml file to see which run they refer to
for sweep_set_dir in sorted(glob.glob(os.path.join(root_dir, '*-*-*/'))):
	sweep_ids = [x for x in os.listdir(sweep_set_dir) if x!='discarded']
	assert len(sweep_ids)==1, f"{len(sweep_ids)} sweeps found in {sweep_set_dir}, unsure which to use"
	sweep_id = sweep_ids[0]
	sweep_set_dir = os.path.join(sweep_set_dir, sweep_id)

	runs_w_validation = set() # runs which have corresponding validation folders present
	val_dirs = [d for d in os.listdir(sweep_set_dir) if re.match(r'^val\d+', d)]
	for val_dir in val_dirs:
		yaml_file = os.path.join(sweep_set_dir, val_dir, 'simple_evaluation_results.yaml')
		if os.path.exists(yaml_file):
			with open(yaml_file, 'r') as f:
				config = yaml.safe_load(f)
				run_name = config.get('model').split('/')[-3]
				if run_name:
					runs_w_validation.add(run_name)

	all_runs = set(x.split('/')[-1] for x in glob.glob(os.path.join(sweep_set_dir, '*-*-*')))
	print(sweep_set_dir)
	print(f'- Found {len(val_dirs)} val directories and {len(all_runs)} runs')

	uncovered_runs = all_runs - runs_w_validation
	if uncovered_runs:
		print(f"- Runs without val directories:")
	for run in uncovered_runs:
		print(f"	- {run}")

/home/wandb-runs/pace-v2/2D_CNP-only-10/avno7wer
- Found 24 val directories and 25 runs
- Runs without val directories:
	- rose-sweep-1
/home/wandb-runs/pace-v2/2D_CNP-only-20/a681wb08
- Found 24 val directories and 25 runs
- Runs without val directories:
	- restful-sweep-1
/home/wandb-runs/pace-v2/real-only-10/gzpeeznn
- Found 49 val directories and 50 runs
- Runs without val directories:
	- jolly-sweep-1
/home/wandb-runs/pace-v2/real-only-100/c4eio2ma
- Found 31 val directories and 31 runs
/home/wandb-runs/pace-v2/real-only-50/647zxr4l
- Found 34 val directories and 34 runs


In [3]:
## Find the k best performing runs for a given sweep set directory
## use the metrics/mAP50(B) value from the metrics entry of the simple_evaluation_results.yaml file in the val dirs
top_k = 3
best_runs_for_set = dict()
for sweep_set_dir in glob.glob(os.path.join(root_dir, '*-*-*/')):
	sweep_ids = [x for x in os.listdir(sweep_set_dir) if x!='discarded']
	assert len(sweep_ids)==1, f"{len(sweep_ids)} sweeps found in {sweep_set_dir}, unsure which to use"
	sweep_id = sweep_ids[0]
	sweep_set_dir = os.path.join(sweep_set_dir, sweep_id)
	
	map50_scores = dict()  # run_name -> mAP50 score
	val_dirs = [d for d in os.listdir(sweep_set_dir) if re.match(r'^val\d+', d)]
	for val_dir in val_dirs:
		yaml_file = os.path.join(sweep_set_dir, val_dir, 'simple_evaluation_results.yaml')
		if os.path.exists(yaml_file):
			with open(yaml_file, 'r') as f:
				config = yaml.safe_load(f)
				mAP50 = config.get('metrics', {}).get('metrics/mAP50(B)', 0)
				run_name = config.get('model').split('/')[-3]
				map50_scores[run_name] = mAP50
	sweep_set_id = sweep_set_dir.split('/')[-1]
	best_runs_for_set[sweep_set_id] = sorted(map50_scores.items(), key=lambda x: x[1], reverse=True)[:3]
	print(f"Best runs in set {sweep_set_dir}:")
	for run, score in best_runs_for_set[sweep_set_id]	:
		print(f" - {run}: {score*100:.2f}%") # print percentage to 2 decimal place

Best runs in set /home/wandb-runs/pace-v2/real-only-50/647zxr4l:
 - smart-sweep-25: 48.75%
 - dashing-sweep-27: 44.87%
 - different-sweep-30: 43.70%
Best runs in set /home/wandb-runs/pace-v2/real-only-100/c4eio2ma:
 - serene-sweep-22: 56.06%
 - ancient-sweep-19: 54.13%
 - rural-sweep-18: 51.99%
Best runs in set /home/wandb-runs/pace-v2/2D_CNP-only-20/a681wb08:
 - soft-sweep-13: 49.45%
 - lucky-sweep-8: 49.37%
 - valiant-sweep-20: 48.55%
Best runs in set /home/wandb-runs/pace-v2/real-only-10/gzpeeznn:
 - resilient-sweep-11: 23.32%
 - vibrant-sweep-9: 21.84%
 - upbeat-sweep-13: 20.63%
Best runs in set /home/wandb-runs/pace-v2/2D_CNP-only-10/avno7wer:
 - noble-sweep-8: 57.58%
 - vibrant-sweep-20: 55.42%
 - flowing-sweep-23: 55.22%


# Re-finetune models for longer duration

In [4]:
sweep_dir = '/home/wandb-runs/pace-v2/real-only-10/gzpeeznn' ## chose a particular sweep

In [5]:
best_runs = best_runs_for_set[sweep_dir.split('/')[-1]]

In [ ]:
for run, old_mAP in best_runs:
	print(f"Re-finetuning {run} with old mAP {old_mAP*100:.2f}%")
	## get args.yaml file 
	args = yaml.safe_load(open(os.path.join(sweep_dir, run, 'args.yaml')))
	args['epochs'] = 100
	args['patience'] = 20
	args['name'] = args['name'] + '_extended'
	args['save_dir'] = os.path.join(sweep_dir, args['name'])
	args['data'] = args['data'].replace('pace-v2-val.yaml', 'pace-v2.yaml') # since we renamed the data files
	args['project'] = sweep_dir
	
	### TEMP
	args['epochs'] = 2
	args['fraction'] = 0.001

	yolo_finetuner = YOLOFinetuner(**args)
	results_train = yolo_finetuner.train_model()
	
	evaluator_args = {
		'run': str(results_train.save_dir),
		'batch': config.get('batch', 32),
		'imgsz': config.get('eval_imgsz', 640),
		'project': args['project'],
		'split': 'test',  # Evaluate on the final test set
		'name': 'val_'+args['name'],
	}
	evaluator = YOLOEvaluator(**evaluator_args)
	val_results = evaluator.evaluate_model()

	print('Basic metrics:')
	print(val_results.get('metrics'))

	break

Re-finetuning resilient-sweep-11 with old mAP 23.32%
YOLO11n summary: 181 layers, 2,624,080 parameters, 0 gradients, 6.6 GFLOPs
(181, 2624080, 0, 6.614336)
New https://pypi.org/project/ultralytics/8.3.176 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.167 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA A10, 22599MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=5, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/data/configs/pace_v2.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.001, freeze=20, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001318285936761404, lrf=0.

wandb: Currently logged in as: vikhyat3 (vikhyat-3-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 'model.3.bn.weight'
Freezing layer 'model.3.bn.bias'
Freezing layer 'model.4.cv1.conv.weight'
Freezing layer 'model.4.cv1.bn.weight'
Freezing layer 'model.4.cv1.bn.bias'
Freezing layer 'model.4.cv2.conv.weight'
Freezing layer 'model.4.cv2.bn.weig

train: Scanning /home/data/pace/toycar_can_v2/train/labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2325.4±1723.8 MB/s, size: 1068.9 KB)


...

                   all       6552       7071   0.000772      0.207   0.000621   0.000404
               toy_car       3522       4049    0.00105      0.248   0.000951   0.000619
                   can       3022       3022   0.000499      0.166   0.000292   0.000189
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to /home/wandb-runs/pace-v2/real-only-10/gzpeeznn/resilient-sweep-11_extended


lr/pg0,█▁
lr/pg1,▁█
lr/pg2,▁█
metrics/mAP50(B),█▁
metrics/mAP50-95(B),█▁
metrics/precision(B),▁█
metrics/recall(B),▁█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▁


Fetching data path from the model YAML configuration in /home/wandb-runs/pace-v2/real-only-10/gzpeeznn/resilient-sweep-11_extended...
Evaluating on dataset: /home/data/configs/pace_v2.yaml with split: test
Ultralytics 8.3.167 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA A10, 22599MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4845.8±806.7 MB/s, size: 1077.3 KB)


val: Scanning /home/data/pace/toycar_can_v2/test/labels.cache... 4341 images, 100 backgrounds, 0 corrupt: 100%|██████████| 4341/4341 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 136/136 [00:29<00:00,  4.61it/s]


                   all       4341       5349    0.00297      0.771    0.00368    0.00249
               toy_car       2259       3367    0.00296      0.587    0.00276    0.00187
                   can       1982       1982    0.00298      0.954    0.00459     0.0031
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to /home/wandb-runs/pace-v2/real-only-10/gzpeeznn/val51
Evaluation Results:
Class Names: {0: 'toy_car', 1: 'can'}
mAP: [  0.0018736   0.0031011]
Number of Detections per Class: [3367 1982]
Number of Detections per Image: [2259 1982]
Results Dictionary: {'metrics/precision(B)': np.float64(0.0029703924048985223), 'metrics/recall(B)': np.float64(0.7705244138140203), 'metrics/mAP50(B)': np.float64(0.003676750025114848), 'metrics/mAP50-95(B)': np.float64(0.002487372109730922), 'fitness': np.float64(0.002606309901269315)}
Speed: {'preprocess': 0.10786274762554285, 'inference': 1.1652014710642424, 'loss': 0.00023716893128619934, 'postpr

# Test the extended models